In [ ]:
%defaultDatasource jdbc:h2:mem:db

# Analisando o Uso de Medicamentos
* Esta tarefa analisará os dados conforme três abordagens: análise do comportamento estatístico; análise de correlação entre pares; análise de correlação de variáveis.
* A partir dos arquivos CSV que estão no diretórios /data/nhanes considere as seguintes tabelas SQL.

In [ ]:
CREATE TABLE Pessoa (
  Id VARCHAR(5) NOT NULL,
  Genero INTEGER,
  Idade INTEGER,
  PRIMARY KEY(Id)
) AS SELECT
    Id,
    Genero,
    Idade
FROM CSVREAD('../../../data/nhanes/demographic-person.csv');

CREATE TABLE Medicamento (
  Codigo VARCHAR(6) NOT NULL,
  NomeMedicamento VARCHAR(100) NOT NULL,
  PRIMARY KEY(Codigo)
) AS SELECT
    Codigo,
    NomeMedicamento
FROM CSVREAD('../../../data/nhanes/medications-drug.csv');

CREATE TABLE UsoMedicamento (
  IdPessoa VARCHAR(5) NOT NULL,
  CodMedicamento VARCHAR(6) NOT NULL,
  DiasUso INTEGER,
  FOREIGN KEY(IdPessoa)
    REFERENCES Pessoa(Id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(CodMedicamento)
    REFERENCES Medicamento(Codigo)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
    IdPessoa,
    CodMedicamento,
    DiasUso
FROM CSVREAD('../../../data/nhanes/medications-use.csv');

In [ ]:
SELECT * FROM Pessoa;
SELECT * FROM Medicamento;
SELECT * FROM UsoMedicamento;

# I. Análise de comportamento estatístico

## 1) Contabilizando o uso de medicamentos

* Liste o nome de todos os medicamentos e a quantidade de pessoas que usa cada um deles.

In [ ]:
SELECT m.NomeMedicamento, COUNT(*) QtdUsuarios
FROM Medicamento m, UsoMedicamento u
WHERE m.Codigo = u.CodMedicamento
GROUP BY u.CodMedicamento

## 2) Medicamento mais usado com VIEW

* Informe o código do medicamento mais usado: fazendo uso de VIEW.

In [ ]:
CREATE VIEW MedxUsuarios AS 
SELECT u.CodMedicamento, COUNT(*) QtdUsuario
FROM UsoMedicamento u
GROUP BY u.CodMedicamento;

CREATE VIEW MaisUsado AS
SELECT MAX(QtdUsuario) MaxQTD 
FROM MedxUsuarios mxd;

SELECT mxd.CodMedicamento 
FROM MedxUsuarios mxd, MaisUsado mu 
WHERE mxd.QtdUsuario = mu.MaxQTD 

## 3) Medicamento mais usado sem VIEW

* Informe o código do medicamento mais usado: sem fazer uso de VIEW.

In [ ]:
SELECT mxd.CodMedicamento
FROM (SELECT u.CodMedicamento, COUNT(*) QtdUsuario
    FROM UsoMedicamento u
    GROUP BY u.CodMedicamento) AS MXD
WHERE mxd.QtdUsuario  
    = (SELECT MAX(QtdUsuario) MaxQTD 
        FROM (SELECT u.CodMedicamento, COUNT(*) QtdUsuario
            FROM UsoMedicamento u
            GROUP BY u.CodMedicamento))

## 4) Nome do medicamento mais usado
* Informe o nome do medicamento mais usado (uso de VIEW é opcional).

In [ ]:
SELECT m.NomeMedicamento 
FROM Medicamento m 
WHERE m.Codigo = (SELECT mxd.CodMedicamento FROM MedxUsuarios mxd, MaisUsado mu WHERE mxd.QtdUsuario = mu.MaxQTD)

## 5) Contabilizando quanto as pessoas usam de medicamento

* Informe o número médio de uso de medicamento por pessoa.

In [ ]:
CREATE VIEW PessoaxMed AS
SELECT u.IdPessoa, COUNT(*) QtdMed
FROM UsoMedicamento u
GROUP BY u.IdPessoa

SELECT AVG(QtdMed) FROM PessoaxMed

## 6) Pessoas que usam mais do que a média

* Liste o id das pessoas que usam mais medicamentos do que a média.

In [ ]:
SELECT pxm.IdPessoa FROM PessoaxMed pxm
WHERE pxm.QtdMed > (SELECT AVG(QtdMed) FROM PessoaxMed)

## 7) Análise do uso de medicamento

* Considere que um médico quer fazer responder a seguinte questão: pessoas tendem a usar mais medicamentos conforme ficam mais velhas?
* Escreva uma query que ajude o médico a realizar esta análise.

In [ ]:
/*Exibe quantas pessoas/idade consomem mais medicamentos do que a média*/
SELECT pxmi.Idade, COUNT(*) QtdPessoasAcimaMédia 
FROM (SELECT pxm.*, p.Idade idade 
    FROM PessoaxMed pxm, Pessoa p
    Where pxm.IdPessoa = p.Id) pxmi
WHERE pxmi.QtdMed > (SELECT AVG(QtdMed) FROM PessoaxMed)
GROUP BY pxmi.Idade

# II. Análise de correlação em pares
## Estudo de ação cruzada entre medicamentos

* Um pesquisador quer estudar o efeito cruzado de usar dois medicamentos simultaneamente. 
* Para isso ele precisará da seguinte sequência de queries.

## 8) Medicamentos tomados em conjunto
* Construa uma VIEW com duas colunas: medicamentoA, medicamentoB
* Nessa VIEW, é criada uma tupla (medicamentoA, medicamentoB) toda vez que uma pessoa toma o medicamento A e também o medicamento B.
* Haverá repetição de tuplas, já que mais de uma pessoa pode tomar o mesmo par de medicamentos.
* Cuidado para não duplicar a mesma informação, ou seja, considerando que uma pessoa tomou o medicamentoX e o medicamentoY, cuidado para não criar duas tuplas: (medicamentoX, medicamentoY) e (medicamentoY, medicamentoX).

In [ ]:
CREATE VIEW InteracaoMedicamento AS SELECT ua.CodMedicamento MedA, ub.CodMedicamento MedB 
FROM (SELECT * FROM UsoMedicamento ORDER BY CodMedicamento) AS ua CROSS JOIN (SELECT * FROM UsoMedicamento ORDER BY CodMedicamento) AS ub
WHERE ua.IdPessoa = ub.IdPessoa AND ua.CodMedicamento <> ub.CodMedicamento AND ua.CodMedicamento < ub.CodMedicamento

SELECT * FROM InteracaoMedicamento

## 9) Total de medicamento tomados em conjunto
* Apresente o total de pessoas que toma cada par de medicamentos.

In [ ]:
SELECT im.MedA, im.MedB, COUNT(*)
FROM InteracaoMedicamento im
GROUP BY MedA, MedB

# III. Análise de correlação entre variáveis
* Consiste na análise da seguinte questão: como a alteração de uma variável afeta a outra.
## Matriz de análise
* Considere a matriz abaixo.

In [ ]:
CREATE TABLE Matriz (
  Id VARCHAR(5) NOT NULL,
  Genero INTEGER,
  Idade INTEGER,
  IdadeAte60 BOOLEAN,
  MaisUmAnoMedicamento BOOLEAN,
  MedicamentosAcimaMedia BOOLEAN,
  PRIMARY KEY(Id)
) AS SELECT
    Id,
    Genero,
    Idade,
    FALSE, FALSE, FALSE
FROM CSVREAD('../../../data/nhanes/demographic-person.csv');

SELECT * FROM Matriz;

## 10) Atualização simples da matriz
* Utilize o comando UPDATE para atualizar a coluna IdadeAte60 da tabela colocando verdadeiro para as pessoas que têm até 60 anos de idade.

In [ ]:
/*Considerado Idade <= 60 = true; Idade > 60 false para o campo IdadeAte60*/
UPDATE Matriz mat SET mat.IdadeAte60 = true
WHERE mat.Idade <= 60

## 11) Atualização vinculada a uma segunda tabela
* Utilize o comando UPDATE para atualizar a coluna MaisUmAnoMedicamento, colocando verdadeiro para aquelas pessoas que usam pelo menos um medicamento há mais de um ano.

In [ ]:
UPDATE Matriz mat SET mat.MaisUmAnoMedicamento = true
WHERE mat.Id IN (SELECT u.IdPessoa FROM UsoMedicamento u WHERE u.DiasUso > 365)

## 12) Atualização baseada em uma consulta complexa
* Utilize o comando UPDATE para atualizar a coluna MedicamentosAcimaMedia, colocando verdadeiro para aquelas pessoas que usam um número de medicamentos acima da média.

In [ ]:
UPDATE Matriz mat SET mat.MedicamentosAcimaMedia = true
WHERE mat.Id IN (SELECT pxm.IdPessoa FROM PessoaxMed pxm WHERE pxm.QtdMed > (SELECT AVG(QtdMed) FROM PessoaxMed))

## 13) Que análise interessante pode ser feita?
* A partir da Matriz criada, proponha uma análise interessante.

### 13a) Descreva a seguir (em Markdown) qual a análise que você propõe.

Propõe-se a seguinte análise:
* Relação de gênero entre a quantidade de pessoas que se supermedicam (tomam multiplos medicamentos) há longo prazo (há mais de um ano)

### 13b) Escreva uma consulta em SQL que fornece informações para a análise proposta.

In [ ]:
SELECT mat.Genero, COUNT(*) nºPessoas
FROM Matriz mat
WHERE mat.MaisUmAnoMedicamento = true AND mat.MedicamentosAcimaMedia = true
GROUP BY mat.Genero